In [1]:
#import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import ftplib
from matplotlib.dates import DateFormatter
import ipywidgets as widgets
%matplotlib widget


In [2]:
#ftp server data
hostname = "ftpupload.net"
username = "epiz_32799455"
pwd = "2RerU5lOOo5s9"
folder = 'shared_files'
filename = 'flight_fares.csv'



#download the file
downloaded_filename = 'downl_'+filename
with ftplib.FTP(hostname, username, pwd) as ftp:
    try:
        ftp.cwd(folder)
        ftp.encoding = "utf-8"
        with open(downloaded_filename, 'wb+') as f:
            ftp.retrbinary("RETR " + filename ,f.write)
            f.seek(0)
            new_string = f.read().decode("utf-8") 
            
    except:
        print ("Error")

#print(new_string)

In [3]:
df_all = pd.read_csv(downloaded_filename, sep = '\t',parse_dates=['Query date'])
df_list_label = [x for _, x in df_all.groupby(df_all['Label'])]
label_list = [df.iloc[0]['Label'] for df in df_list_label]
df_list_ID = [[x for _, x in df.groupby(df['ID'])] for df in df_list_label] 


    

In [4]:


check_list = [widgets.Checkbox(value=False, description=lab, indent=False) for lab in label_list]
check_list[-1].value = True

display(widgets.GridBox(check_list, layout=widgets.Layout(grid_template_columns="repeat(3, 200px)")))

fig,ax = plt.subplots(figsize = [8,4])

def plot_sel(change):
    ax.clear()
    for ii, df_list in enumerate(df_list_ID):
        if check_list[ii].value:
            for df in df_list:
                tmp=df.iloc[0]
                flight_descript = f"{tmp['Label']} {tmp['Origin']}-{tmp['Destination']} {tmp['Date']}"
                ax.plot(df['Query date'],df['Price'],'.-',label=flight_descript)
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.grid('True')
    ax.set_xlabel('Query date')
    ax.set_ylabel('Price (€)')
    ax.xaxis.set_major_formatter(DateFormatter('%d/%m'))
    fig.tight_layout()



plot_sel(None)
[cl.observe(plot_sel) for cl in check_list]

None


GridBox(children=(Checkbox(value=False, description='Navidad', indent=False), Checkbox(value=True, description…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …